In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import _pickle as cPickle

In [2]:
def unpickle(filename):
    fo = open(filename, 'rb')
    fdict = cPickle.load(fo, encoding='latin1')
    fo.close()
    return fdict

In [3]:
data_dict = {}
for i in range(1, 6):
    data_dict['data_batch_' + str(i)] = unpickle('cucumber_data/p1/data_batch_' + str(i))
data_dict['test_batch'] = unpickle('cucumber_data/p1/test_batch')

In [4]:
def get_images(data, labels):
    num_images = data.shape[0] // 3
    images = np.zeros((num_images, 32, 32, 3))
    labels_arr = np.zeros((num_images, 9))
    for i in range(num_images):
        images[i] = np.vstack((data[3*i], data[3*i+1], data[3*i+2])).reshape(3, 32, 32).transpose(1,2,0) / 255
        labels_arr[i][labels[i]] = 1
    return (images, labels_arr)

In [5]:
images_dict = {}
labels_dict = {}
for data_batch in data_dict:
    images_dict[data_batch], labels_dict[data_batch] = get_images(data_dict[data_batch]['data'], data_dict[data_batch]['labels'])

In [6]:
validation_images = images_dict['data_batch_5']
validation_labels = labels_dict['data_batch_5']

test_images = images_dict['test_batch']
test_labels = labels_dict['test_batch']

train_images = np.vstack((images_dict['data_batch_1'], 
                          images_dict['data_batch_2'], 
                          images_dict['data_batch_3'], 
                          images_dict['data_batch_4']))

train_labels = np.vstack((labels_dict['data_batch_1'], 
                          labels_dict['data_batch_2'], 
                          labels_dict['data_batch_3'], 
                          labels_dict['data_batch_4']))

In [7]:
def get_batch(images, batch_size):
    images_ind = list(range(len(images)))
    random.shuffle(images_ind)
    return images[images_ind[:batch_size]]

In [8]:
def leaky_relu(x, alpha=0.01):
    return tf.maximum(alpha*x, x)

## Convolutional Neural Network

In [9]:
input_size = 32*32*3
filter_conv1 = 32
filter_conv2 = 64
#neuron_1 = 4*4*filter_conv2
neuron_1 = 8*8*filter_conv2
neuron_2 = neuron_1

learning_rate = 1e-4
num_train_examples = train_images.shape[0]
iters = 17*num_train_examples
batch_size = 200
num_subjects = 9
keep_prob = tf.placeholder(tf.float32)

In [10]:
x = tf.placeholder(shape=[None, 32, 32, 3], dtype=tf.float32, name='images')
y_correct = tf.placeholder(shape=[None, num_subjects], dtype=tf.float32, name='correct_output')

# first convolution
W_conv1 = tf.Variable(tf.truncated_normal(shape=[3, 3, 3, filter_conv1], stddev=0.1), name="W_conv1")
b_conv1 = tf.Variable(tf.zeros([filter_conv1]), name="b_conv1")
h_conv1 = tf.nn.conv2d(x, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1   # (None, 32, 32, filter_conv1)
#h_conv1_relu = tf.nn.relu(h_conv1)
h_conv1_relu = leaky_relu(h_conv1)

# first max pool
h_pool_1 = tf.nn.max_pool(h_conv1_relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  # (None, 16, 16, filter_conv1)

# second convolution
W_conv2 = tf.Variable(tf.truncated_normal(shape=[4, 4, filter_conv1, filter_conv2], stddev=0.1), name="W_conv2")
b_conv2 = tf.Variable(tf.zeros([filter_conv2]), name="b_conv2")
h_conv2 = tf.nn.conv2d(h_pool_1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2   # (None, 16, 16, filter_conv2)
#h_conv2_relu = tf.nn.relu(h_conv2)
h_conv2_relu = leaky_relu(h_conv2)


# second max pool
h_pool_2 = tf.nn.max_pool(h_conv2_relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  # (None, 8, 8, filter_conv2)


In [11]:
# flatten
fc_input = tf.reshape(h_pool_2, [-1, neuron_1])

# Fully Connected
W_1 = tf.Variable(tf.truncated_normal([neuron_1, neuron_2]), name="W_1")
b_1 = tf.Variable(0.0, [neuron_2], name="b_1")
a_1 = tf.nn.relu(tf.matmul(fc_input, W_1) + b_1)

keep_prob = tf.placeholder(tf.float32)
a_1_drop = tf.nn.dropout(a_1, keep_prob)

W_2 = tf.Variable(tf.truncated_normal([neuron_2, num_subjects]), name="W_2")
b_2 = tf.Variable(0.0, [num_subjects], name="b_2")
y = tf.matmul(a_1_drop, W_2) + b_2

In [12]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_correct, logits=y))

# Optimization
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Evaluate model
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_correct,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

validation_feed_dict = {x: validation_images, y_correct: validation_labels, keep_prob: 1.0 }
test_feed_dict = {x: test_images, y_correct: test_labels, keep_prob: 1.0}
for i in range(iters):
    train_indices = random.sample(range(num_train_examples), batch_size)
    train_feed_dict = {x: train_images[train_indices], y_correct: train_labels[train_indices], keep_prob:0.4 }

    if i%1000 == 0:
        train_accuracy = accuracy.eval(feed_dict=train_feed_dict)
        print("step %d, training accuracy %g"%(i, train_accuracy))
        print("Accuracy validation: {}".format(accuracy.eval(
                feed_dict = validation_feed_dict)))
    _ = sess.run(train_step, feed_dict = train_feed_dict)

print("DONE. Test accuracy: {}".format(accuracy.eval(feed_dict = test_feed_dict)))

step 0, training accuracy 0.1
Accuracy validation: 0.10303031653165817
step 1000, training accuracy 0.455
Accuracy validation: 0.6161617040634155
step 2000, training accuracy 0.635
Accuracy validation: 0.6848485469818115
step 3000, training accuracy 0.715
Accuracy validation: 0.7171717882156372
step 4000, training accuracy 0.775
Accuracy validation: 0.7353535890579224
step 5000, training accuracy 0.795
Accuracy validation: 0.7333333492279053
step 6000, training accuracy 0.835
Accuracy validation: 0.7575757503509521
step 7000, training accuracy 0.875
Accuracy validation: 0.7777777910232544
step 8000, training accuracy 0.865
Accuracy validation: 0.8141414523124695
step 9000, training accuracy 0.885
Accuracy validation: 0.8161616921424866
step 10000, training accuracy 0.91
Accuracy validation: 0.8323233127593994
step 11000, training accuracy 0.905
Accuracy validation: 0.8161616325378418
step 12000, training accuracy 0.94
Accuracy validation: 0.8242424726486206
step 13000, training accurac